# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avera,33.1940,-82.5271,15.86,84,100,2.57,US,1675300695
1,1,mataura,-46.1927,168.8643,26.46,52,100,0.81,NZ,1675300697
2,2,busselton,-33.6500,115.3333,19.56,51,92,7.62,AU,1675300704
3,3,juneau,58.3019,-134.4197,0.68,94,100,1.54,US,1675300688
4,4,bud,39.4470,-86.1758,-1.99,61,100,1.04,US,1675300706


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 18) & 
    (city_data_df["Max Temp"] <= 26) & 
    (city_data_df["Humidity"] >= 10) & 
    (city_data_df["Humidity"] <= 50) &
    (city_data_df["Cloudiness"] <= 50) &
    (city_data_df["Wind Speed"] <= 5)
]

# Drop any rows with null values
ideal_df = ideal_df.dropna(how="any")

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,raga,8.4596,25.6780,22.45,21,0,2.17,SS,1675300573
168,168,filingue,14.3521,3.3168,23.70,16,4,3.73,NE,1675300872
259,259,bako,5.7833,36.5667,20.11,47,45,1.79,ET,1675300948
266,266,lat yao,15.7510,99.7893,23.49,39,9,1.19,TH,1675300952
324,324,nioro,13.3500,-15.7500,23.07,20,5,4.50,GM,1675300994
339,339,canchungo,12.0672,-16.0333,20.62,44,2,3.01,GW,1675301013
511,511,bara,10.3744,10.7288,22.10,26,3,4.30,NG,1675301165
547,547,ratnagiri,16.9833,73.3000,23.74,50,6,2.97,IN,1675301204


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City","Lat","Lng","Country","Humidity"]].copy().reset_index()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,index,City,Lat,Lng,Country,Humidity,Hotel Name
0,115,raga,8.4596,25.6780,SS,21,
1,168,filingue,14.3521,3.3168,NE,16,
2,259,bako,5.7833,36.5667,ET,47,
3,266,lat yao,15.7510,99.7893,TH,39,
4,324,nioro,13.3500,-15.7500,GM,20,
5,339,canchungo,12.0672,-16.0333,GW,44,
6,511,bara,10.3744,10.7288,NG,26,
7,547,ratnagiri,16.9833,73.3000,IN,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": f"accommodation.hotel",
    "limit": 1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
raga - nearest hotel: No hotel found
filingue - nearest hotel: No hotel found
bako - nearest hotel: Goh Hotel
lat yao - nearest hotel: resort
nioro - nearest hotel: No hotel found
canchungo - nearest hotel: Hotel Pereira
bara - nearest hotel: No hotel found
ratnagiri - nearest hotel: Vihar


,index,City,Lat,Lng,Country,Humidity,Hotel Name
0,115,raga,8.4596,25.6780,SS,21,No hotel found
1,168,filingue,14.3521,3.3168,NE,16,No hotel found
2,259,bako,5.7833,36.5667,ET,47,Goh Hotel
3,266,lat yao,15.7510,99.7893,TH,39,resort
4,324,nioro,13.3500,-15.7500,GM,20,No hotel found
5,339,canchungo,12.0672,-16.0333,GW,44,Hotel Pereira
6,511,bara,10.3744,10.7288,NG,26,No hotel found
7,547,ratnagiri,16.9833,73.3000,IN,50,Vihar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.75,
    color = "City",
    hover_cols = ["Lng","Lat","City","Humidity","Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)